# DenseNet-169 for Multi-class Classification of CXR Images

### Objectives

The aim of this notebook is to train variations of the DenseNet arhictecture on the Chest X-ray (Pneumonia) data set sourced from Kaggle. The images are composed of CXR scans representing normal lungs, bacterial pneumonia, and viral pneumonia.

A variable classification head will be built on top of the chosen network, in which varying hyperparameters will be configured and evaluated. The experiment tracking will be conducted with the *Weights and Biases* platform. The optimal model will be selected based on a trade-off evaluation between **MCC score** and **validation loss**. 

### Machine Configurations

`GPU` : NVIDIA GeForce RTX 4090;
`CPU` : AMD Ryzen 7 3700X 8-Core Processor;
`VRAM` : 24.0 GB

### Load all required libraries:

In [1]:
import os
import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import f1_score, matthews_corrcoef

import tensorflow as tf

# Suppress TensorFlow messages:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print("TensorFlow version : ", tf.__version__)  # check TensorFlow version
gpu = tf.config.list_physical_devices("GPU")    # check if TensorFlow is using the GPU
print(gpu)

# Enable memory growth for GPU:
try:
   tf.config.experimental.set_memory_growth(gpu[0], True)
   print("Memory growth set.")
except:
    print("GPU runtime already initialised.")

TensorFlow version :  2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth set.


### Configure experiment:

In [2]:
# Configure notebook name for WandB:
%env "WANDB_NOTEBOOK_NAME" "PCXR-evaluating-pretrained-cnn-model"
import wandb
from wandb.keras import WandbMetricsLogger
wandb.login()

env: "WANDB_NOTEBOOK_NAME"="PCXR-evaluating-pretrained-cnn-model"


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marcus-lim. Use `wandb login --relogin` to force relogin


True

In [3]:
# Choose method for hyperparameter selection:
sweep_config = {
    "method": "bayes"
    }

In [4]:
# Establish objective to optimise:
metric = {
    "name": "val_loss",
    "goal": "minimize"   
    }

sweep_config["metric"] = metric

In [5]:
# Choose hyperparameters to sweep:
param_dict = {
    "optimiser": {
        "values": ["adam", "sgd"]
        },
    "fc_layer_size_1": {
        "values": [16, 32, 64, 128, 256, 512]
        },
    "fc_layer_size_2": {
        "values": [16, 32, 64, 128, 256, 512]
        },
    "dropout_1": {
        "values": [0.2, 0.6, 0.8]
        },
    "dropout_2": {
        "values": [0.2, 0.6, 0.8]
        },
    "dropout_3": {
        "values": [0.2, 0.6, 0.8]
        },
    }

sweep_config["parameters"] = param_dict

In [6]:
# Instantiate constant value for epoch:
param_dict.update({
    "epochs": {
        "value": 10},
    "batch_size": {
        "value": 32}
    })

In [7]:
# Use uniform distribution for learning rate:
param_dict.update({
    "learning_rate": {
        "distribution": "uniform",
        "min": 0.0001,
        "max": 0.005
      }
    })

In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'value': 32},
                'dropout_1': {'values': [0.2, 0.6, 0.8]},
                'dropout_2': {'values': [0.2, 0.6, 0.8]},
                'dropout_3': {'values': [0.2, 0.6, 0.8]},
                'epochs': {'value': 10},
                'fc_layer_size_1': {'values': [16, 32, 64, 128, 256, 512]},
                'fc_layer_size_2': {'values': [16, 32, 64, 128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.005,
                                  'min': 0.0001},
                'optimiser': {'values': ['adam', 'sgd']}}}


### Load data:

In [9]:
def build_datasets(train_dir="..\\artifacts\\train",
                   test_dir="..\\artifacts\\test",
                   class_names=["normal", "bacteria", "virus"],
                   image_size=(300, 300),
                   batch_size=32,
                   val_split=0.2):
    
    train_ds, valid_ds = tf.keras.utils.image_dataset_from_directory(
        directory=train_dir,
        label_mode="int",
        class_names=class_names,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True,
        seed=42,
        validation_split=val_split,
        subset="both")

    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=test_dir,
        label_mode="int",
        class_names=class_names,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True,
        seed=42,
        subset=None)

    return (train_ds, valid_ds, test_ds)

### Construct network:

In [10]:
def build_base_model(img_shape=(300, 300, 3), network="densenet-169"):

    if network == "densenet-121":
        return tf.keras.applications.densenet.DenseNet121(
            input_shape=img_shape,
            include_top=False,
            weights="imagenet")
    
    if network == "densenet-169":
        return tf.keras.applications.densenet.DenseNet169(
            input_shape=img_shape,
            include_top=False,
            weights="imagenet")
    
    if network == "densenet-201":
        return tf.keras.applications.densenet.DenseNet201(
            input_shape=img_shape,
            include_top=False,
            weights="imagenet")

In [20]:
def build_network(network="densenet-169", fc_layer_size_1=64, fc_layer_size_2=64, 
                  dropout_1=0.2, dropout_2=0.2, dropout_3=0.2):

    # Construct base model (default is DenseNet-169)
    IMG_SHAPE = (300, 300, 3)
    base_model = build_base_model(img_shape=IMG_SHAPE, network=network)

    # Freeze convolutional base
    base_model.trainable = False

    inputs = tf.keras.Input(shape=IMG_SHAPE)

    # Add an augmentation layer for additional training data
    augment_layer = tf.keras.Sequential([
        tf.keras.layers.RandomContrast(factor=0.2, seed=42),
        tf.keras.layers.RandomBrightness(factor=0.2, seed=42),
        tf.keras.layers.RandomRotation(factor=0.01, seed=42)
    ])

    # Establish base model architecture:
    x = augment_layer(inputs)
    x = tf.keras.applications.densenet.preprocess_input(x)
    x = base_model(x, training=False)    # Prevent training of BN layers

    # Add custom classification head:
    x = tf.keras.layers.MaxPooling2D(input_shape=(1664,))(x)
    x = tf.keras.layers.Dropout(dropout_1)(x)  # Prevent overfitting
    x = tf.keras.layers.Dense(fc_layer_size_1, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_2)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(dropout_3)(x)

    x = tf.keras.layers.Dense(fc_layer_size_2, activation="relu")(x)
    outputs = tf.keras.layers.Dense(3, activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)

### Define optimiser:

In [21]:
def build_optimiser(learning_rate=0.0001, optimiser="adam"):
    if optimiser.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=learning_rate)
    if optimiser.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=learning_rate)

### Create callback to log custom metrics:

In [13]:
# Compute MCC, F1, and AUC scores and log in WandB:
class CustomLogCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, x_val, y_val):
        super().__init__()
        self.model = model
        self.x_val = x_val
        self.y_val = y_val

        # Instantiate metrics:
        self._mcc = tf.keras.metrics.Mean(name="mcc")
        self._f1score = tf.keras.metrics.Mean(name="f1_score")
        self._aucroc = tf.keras.metrics.AUC(name="auc_roc")

        self.epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        self.epoch += 1

        # Reset values to 0 for each metric:
        self._mcc.reset_state()
        self._f1score.reset_state()
        self._aucroc.reset_state()

        print("Generating predictions and computing metrics for Epoch {} ".format(self.epoch))
        predictions = self.model.predict(self.x_val)

        # Compute values for each metric:
        f1score = f1_score(self.y_val, np.argmax(predictions, axis=-1),
                           average=None)
        mcc = matthews_corrcoef(self.y_val, np.argmax(predictions, axis=-1))

        # Update each metric with respective values:
        self._mcc.update_state(mcc)
        self._f1score.update_state(f1score)
        self._aucroc.update_state(self.y_val, np.argmax(predictions, axis=-1))

        print("training loss : {} , training acc : {} , mcc score : {}".format(
            logs["loss"], logs["accuracy"], self._mcc.result().numpy()
        ))
        print("aucroc score  : {} , f1 score     : {} ".format(
            self._aucroc.result().numpy(), self._f1score.result().numpy()
        ))

        # Log metrics to WandB:
        wandb.log({"mcc": self._mcc.result().numpy(),
                   "fmeasure": self._f1score.result().numpy(),
                   "auc_roc": self._aucroc.result().numpy()})

### Create training function:

In [22]:
# Build data sets outside of train function:
train_ds, valid_ds, _ = build_datasets(batch_size=None)

# Convert validation data to arrays for custom metric computation:
x_val = np.reshape(np.concatenate([x for x, _ in valid_ds], axis=0), (936, 300, 300, 3))
y_val = np.concatenate(np.reshape([y for _, y in valid_ds], (1, 936)), axis=0)

def train(model, train_dir="..\\artifacts\\train", batch_size=32, 
          epochs=10, learning_rate=0.0001, optimiser_name="adam"):

    tf.keras.backend.clear_session()
    train_ds, valid_ds, _ = build_datasets(train_dir=train_dir, batch_size=batch_size)

    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    valid_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)
    
    optimiser = build_optimiser(learning_rate=learning_rate, optimiser=optimiser_name)

    # Compile model:
    model.compile(optimizer=optimiser,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=["accuracy"])
    
    # Fit model:
    # Configure callbacks:
    callbacks = [CustomLogCallback(model, x_val, y_val),
                 wandb.keras.WandbMetricsLogger()]
                 
    # Train model:
    model.fit(train_ds, 
              epochs=epochs,
              validation_data=valid_ds, 
              callbacks=callbacks, 
              verbose=0)

Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.


# **DenseNet-169 Sweep (unenhanced):**

### Create sweep training function:

In [19]:
train_dir = "..\\artifacts\\train"
network = "densenet-169"

def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(network=network,
                              fc_layer_size_1=wandb.config.fc_layer_size_1,
                              fc_layer_size_2=wandb.config.fc_layer_size_2,
                              dropout_1=wandb.config.dropout_1,
                              dropout_2=wandb.config.dropout_2,
                              dropout_3=wandb.config.dropout_3)

        train(model=model,
              train_dir=train_dir,
              batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, 
              learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)

### Initialise sweep and run agent:

In [17]:
sweep_id = wandb.sweep(sweep_config, project="PCXR-DenseNet-169-Evaluation")

Create sweep with ID: 8ib2hshg
Sweep URL: https://wandb.ai/marcus-lim/PCXR-DenseNet-169-Evaluation/sweeps/8ib2hshg


In [18]:
wandb.agent(sweep_id, function=sweep_train, count=25)

wandb: Agent Starting Run: orybdmqy with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.8
wandb: 	dropout_2: 0.8
wandb: 	dropout_3: 0.8
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 256
wandb: 	fc_layer_size_2: 256
wandb: 	learning_rate: 0.0012589019710188153
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 71ms/step
training loss : 1.0229718685150146 , training acc : 0.5448238849639893 , mcc score : 0.581060528755188
aucroc score  : 0.8908283710479736 , f1 score     : 0.658346951007843 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 70ms/step
training loss : 0.8477164506912231 , training acc : 0.6200640201568604 , mcc score : 0.6144558787345886
aucroc score  : 0.9071005582809448 , f1 score     : 0.7198393940925598 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 69ms/step
training loss : 0.7462443709373474 , training acc : 0.6707577109336853 , mcc score : 0.6158942580223083
aucroc score  : 0.9093195199966431 , f1 score     : 0.72578287124

auc_roc,▁▆▆▅▆▅▇██▇
epoch/accuracy,▁▄▆▆▇▇▇▇▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▃▂▂▁▁▁▁
epoch/val_accuracy,▁▆▆▆▅▆▇██▇
epoch/val_loss,█▅▄▅▅▄▂▂▁▁
fmeasure,▁▆▇▇▇▇▇███
mcc,▁▅▆▆▆▆▇██▇
auc_roc,0.91065
epoch/accuracy,0.73693


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: zpk07j50 with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.2
wandb: 	dropout_2: 0.6
wandb: 	dropout_3: 0.8
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 256
wandb: 	fc_layer_size_2: 32
wandb: 	learning_rate: 0.0024459066191745613
wandb: 	optimiser: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 72ms/step
training loss : 1.2755179405212402 , training acc : 0.41462114453315735 , mcc score : 0.5366719365119934
aucroc score  : 0.8825443387031555 , f1 score     : 0.6647999882698059 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 69ms/step
training loss : 0.939624547958374 , training acc : 0.5715048313140869 , mcc score : 0.5770829916000366
aucroc score  : 0.8914201259613037 , f1 score     : 0.6597205400466919 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 70ms/step
training loss : 0.8756446242332458 , training acc : 0.6192635893821716 , mcc score : 0.6019359827041626
aucroc score  : 0.9017751812934875 , f1 score     : 0.695249140

...

Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 71ms/step
training loss : 0.6254337430000305 , training acc : 0.7537353038787842 , mcc score : 0.6103020906448364
aucroc score  : 0.9124260544776917 , f1 score     : 0.7355310320854187 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 67ms/step
training loss : 0.5223193764686584 , training acc : 0.7852187752723694 , mcc score : 0.6616048812866211
aucroc score  : 0.9366863965988159 , f1 score     : 0.7481279969215393 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 67ms/step
training loss : 0.48618364334106445 , training acc : 0.8022945523262024 , mcc score : 0.6723199486732483
aucroc score  : 0.9374260306358337 , f1 score     : 0.77618384

auc_roc,▁▇▇▆█▅▆▇▇▆
epoch/accuracy,▁▄▅▆▆▆▆▇▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▃▂▂▂▂▁▁
epoch/val_accuracy,▁▆▆██▂▇▆▇█
epoch/val_loss,▆▇▂▁▂█▂▂▂▁
fmeasure,▁▃▇▇▇▄▆▃▆█
mcc,▁▆▇██▅▇▆▇█
auc_roc,0.9324
epoch/accuracy,0.83858


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: hxlum7ow with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.2
wandb: 	dropout_2: 0.6
wandb: 	dropout_3: 0.6
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 256
wandb: 	fc_layer_size_2: 256
wandb: 	learning_rate: 0.003592854906298806
wandb: 	optimiser: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 69ms/step
training loss : 0.9747685194015503 , training acc : 0.5373532772064209 , mcc score : 0.5462163090705872
aucroc score  : 0.8795857429504395 , f1 score     : 0.649387538433075 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 68ms/step
training loss : 0.7856658697128296 , training acc : 0.6640875339508057 , mcc score : 0.6121896505355835
aucroc score  : 0.9078402519226074 , f1 score     : 0.7276062965393066 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 68ms/step
training loss : 0.718073308467865 , training acc : 0.6990394592285156 , mcc score : 0.6241486668586731
aucroc score  : 0.9107987880706787 , f1 score     : 0.73688554763

auc_roc,▁▅▅▆▆▇████
epoch/accuracy,▁▅▆▇▇▇████
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▃▂▂▁▁▁▁
epoch/val_accuracy,▁▅▆▆▆▇▇▇▇█
epoch/val_loss,█▅▃▃▂▂▂▂▁▁
fmeasure,▁▆▆▇▇▇▇▇██
mcc,▁▅▅▆▆▇▇▇▇█
auc_roc,0.92692
epoch/accuracy,0.76387


### Check best parameters:

In [34]:
sweep_name = "PCXR-DenseNet-169-Evaluation/sweeps/8ib2hshg"
best_run_169_normal = wandb.Api().sweep(sweep_name).best_run(order="mcc")
opt_params_169_normal = best_run_169_normal.config

wandb: Sorting runs by -summary_metrics.mcc


{'epochs': 10,
 'dataset': 'Chest X-Ray Images (Pneumonia)',
 'dropout_1': 0.2,
 'dropout_2': 0.6,
 'dropout_3': 0.8,
 'optimiser': 'adam',
 'batch_size': 32,
 'num_classes': 3,
 'architecture': 'DenseNet',
 'learning_rate': 0.0017536576826685398,
 'fc_layer_size_1': 256,
 'fc_layer_size_2': 256}

# **DenseNet-169 Sweep (CLAHE-enhanced):**

### Create sweep training function:

In [37]:
train_dir = "..\\enhanced\\train"
network = "densenet-169"

def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(network=network,
                              fc_layer_size_1=wandb.config.fc_layer_size_1,
                              fc_layer_size_2=wandb.config.fc_layer_size_2,
                              dropout_1=wandb.config.dropout_1,
                              dropout_2=wandb.config.dropout_2,
                              dropout_3=wandb.config.dropout_3)

        train(model=model,
              train_dir=train_dir,
              batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, 
              learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)

### Initialise sweep and run agent:

In [38]:
sweep_id = wandb.sweep(sweep_config, project="PCXR-DenseNet-169-Evaluation")

Create sweep with ID: tzhu0pmo
Sweep URL: https://wandb.ai/marcus-lim/PCXR-DenseNet-169-Evaluation/sweeps/tzhu0pmo


In [39]:
wandb.agent(sweep_id, function=sweep_train, count=15)

wandb: Agent Starting Run: xz0m08nz with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.8
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.8
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 32
wandb: 	fc_layer_size_2: 16
wandb: 	learning_rate: 0.0031183376245970704
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 72ms/step
training loss : 0.9000195264816284 , training acc : 0.5856456756591797 , mcc score : 0.4360939562320709
aucroc score  : 0.7494083642959595 , f1 score     : 0.5672735571861267 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 69ms/step
training loss : 0.7479059100151062 , training acc : 0.6688900589942932 , mcc score : 0.4421291947364807
aucroc score  : 0.6653846502304077 , f1 score     : 0.583459198474884 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 70ms/step
training loss : 0.6996587514877319 , training acc : 0.6977054476737976 , mcc score : 0.5173141360282898
aucroc score  : 0.7677514553070068 , f1 score     : 0.6638709902

auc_roc,▄▁▄▅▄▇▇█▆▇
epoch/accuracy,▁▅▆▇▇▇▇▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▃▂▂▂▁▁▁
epoch/val_accuracy,▁▅▅▇▇▇█▇██
epoch/val_loss,█▄▂▂▂▁▁▂▁▁
fmeasure,▁▂▅▆▅▇▇█▆█
mcc,▁▁▄▅▄▇▇█▆▇
auc_roc,0.86331
epoch/accuracy,0.73879


wandb: Agent Starting Run: kvouijuz with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.6
wandb: 	dropout_3: 0.6
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 512
wandb: 	fc_layer_size_2: 64
wandb: 	learning_rate: 0.0007202837874153242
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 73ms/step
training loss : 0.7305593490600586 , training acc : 0.6843649744987488 , mcc score : 0.4697391092777252
aucroc score  : 0.6857988238334656 , f1 score     : 0.6062870025634766 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 70ms/step
training loss : 0.59136563539505 , training acc : 0.7489327788352966 , mcc score : 0.5339622497558594
aucroc score  : 0.7704142928123474 , f1 score     : 0.6726648807525635 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 70ms/step
training loss : 0.5674744844436646 , training acc : 0.7566702365875244 , mcc score : 0.6111197471618652
aucroc score  : 0.8594674468040466 , f1 score     : 0.73687881231

...

Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 73ms/step
training loss : 1.0952242612838745 , training acc : 0.4445037245750427 , mcc score : 0.03321193903684616
aucroc score  : 0.5 , f1 score     : 0.2944854497909546 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 70ms/step
training loss : 1.0777816772460938 , training acc : 0.4455709755420685 , mcc score : 0.04400097578763962
aucroc score  : 0.5 , f1 score     : 0.2817395031452179 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 70ms/step
training loss : 1.0763601064682007 , training acc : 0.4471718370914459 , mcc score : 0.04756230115890503
aucroc score  : 0.5 , f1 score     : 0.26143380999565125 
Generating predictions and compu

auc_roc,▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▁▂▃▅▄▆███
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▅▄▃▃▂▁▁
epoch/val_accuracy,▁▃▆▆▇▇▇███
epoch/val_loss,█▇▆▅▄▃▃▂▂▁
fmeasure,█▇▄▃▂▁▁▁▁▁
mcc,▂▅▅▅▄▁▄██▆
auc_roc,0.5
epoch/accuracy,0.47972


wandb: Agent Starting Run: wja96sqf with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.6
wandb: 	dropout_3: 0.8
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 64
wandb: 	fc_layer_size_2: 32
wandb: 	learning_rate: 0.003838503272791841
wandb: 	optimiser: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 4s 70ms/step
training loss : 1.1813125610351562 , training acc : 0.38393810391426086 , mcc score : 0.4131636321544647
aucroc score  : 0.8019230365753174 , f1 score     : 0.53533536195755 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 67ms/step
training loss : 1.0543301105499268 , training acc : 0.47732123732566833 , mcc score : 0.3731120526790619
aucroc score  : 0.7190828323364258 , f1 score     : 0.4509327709674835 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 67ms/step
training loss : 0.9654155373573303 , training acc : 0.5373532772064209 , mcc score : 0.39093273878097534
aucroc score  : 0.7341715097427368 , f1 score     : 0.49231258

auc_roc,▆▁▂▃▅▅▆▇██
epoch/accuracy,▁▄▅▆▆▇▇███
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▃▃▂▂▁▁▁
epoch/val_accuracy,▂▁▃▅▆▆▇███
epoch/val_loss,█▆▅▄▃▂▂▂▁▁
fmeasure,▆▁▃▅▅▆▇█▇█
mcc,▃▁▂▃▅▅▆▇▇█
auc_roc,0.83905
epoch/accuracy,0.63207


In [41]:
sweep_name = "PCXR-DenseNet-169-Evaluation/sweeps/tzhu0pmo"
best_run_169_clahe = wandb.Api().sweep(sweep_name).best_run(order="mcc")
opt_params_169_clahe = best_run_169_clahe.config
print(opt_params_169_clahe)

wandb: Sorting runs by -summary_metrics.mcc


{'epochs': 10, 'dataset': 'Chest X-Ray Images (Pneumonia)', 'dropout_1': 0.6, 'dropout_2': 0.2, 'dropout_3': 0.8, 'optimiser': 'adam', 'batch_size': 32, 'num_classes': 3, 'architecture': 'DenseNet', 'learning_rate': 0.0039532139179563825, 'fc_layer_size_1': 256, 'fc_layer_size_2': 512}


# **DenseNet-121 Sweep (unenhanced):**

### Create sweep training function:

In [42]:
train_dir = "..\\artifacts\\train"
network = "densenet-121"

def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(network=network,
                              fc_layer_size_1=wandb.config.fc_layer_size_1,
                              fc_layer_size_2=wandb.config.fc_layer_size_2,
                              dropout_1=wandb.config.dropout_1,
                              dropout_2=wandb.config.dropout_2,
                              dropout_3=wandb.config.dropout_3)

        train(model=model,
              train_dir=train_dir,
              batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, 
              learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)

### Initialise sweep and run agent:

In [43]:
sweep_id = wandb.sweep(sweep_config, project="PCXR-DenseNet-121-Evaluation")

Create sweep with ID: jxdr1xzz
Sweep URL: https://wandb.ai/marcus-lim/PCXR-DenseNet-121-Evaluation/sweeps/jxdr1xzz


In [44]:
wandb.agent(sweep_id, function=sweep_train, count=15)

wandb: Agent Starting Run: vjw23h8y with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.8
wandb: 	dropout_3: 0.2
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 16
wandb: 	fc_layer_size_2: 32
wandb: 	learning_rate: 0.004918367455482324
wandb: 	optimiser: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


29084464/29084464 [==============================] - 3s 0us/step
Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 3s 63ms/step
training loss : 1.1423308849334717 , training acc : 0.3292422592639923 , mcc score : 0.28604450821876526
aucroc score  : 0.7828402519226074 , f1 score     : 0.48401129245758057 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 59ms/step
training loss : 1.0350245237350464 , training acc : 0.4573105573654175 , mcc score : 0.40780484676361084
aucroc score  : 0.8276627063751221 , f1 score     : 0.5583050847053528 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 60ms/step
training loss : 0.9712847471237183 , training acc : 0.5469583868980408 , mcc score : 0.445644557476

auc_roc,▁▄▃▅▆▆▇███
epoch/accuracy,▁▄▆▆▇▇████
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▁▁
epoch/val_accuracy,▁▅▆▇▇▇▇███
epoch/val_loss,█▇▆▄▄▃▂▂▁▁
fmeasure,▁▅▄▅▄▅▆▆▆█
mcc,▁▄▅▆▇▇▇███
auc_roc,0.89985
epoch/accuracy,0.65261


wandb: Agent Starting Run: tcfmhdub with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.8
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 64
wandb: 	fc_layer_size_2: 16
wandb: 	learning_rate: 0.0012376997304229006
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 3s 59ms/step
training loss : 0.9412326812744141 , training acc : 0.5621665120124817 , mcc score : 0.5948848128318787
aucroc score  : 0.9229289889335632 , f1 score     : 0.6468175649642944 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 60ms/step
training loss : 0.7150144577026367 , training acc : 0.6910352110862732 , mcc score : 0.6218174695968628
aucroc score  : 0.9142012000083923 , f1 score     : 0.7165684700012207 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 59ms/step
training loss : 0.6559638977050781 , training acc : 0.720917820930481 , mcc score : 0.625676155090332
aucroc score  : 0.9065088629722595 , f1 score     : 0.73059439659

...

Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 3s 59ms/step
training loss : 1.232548475265503 , training acc : 0.37673425674438477 , mcc score : 0.06460757553577423
aucroc score  : 0.5241124033927917 , f1 score     : 0.2679474353790283 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 58ms/step
training loss : 1.1608966588974 , training acc : 0.41969048976898193 , mcc score : 0.07753854244947433
aucroc score  : 0.5195266604423523 , f1 score     : 0.25348395109176636 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 57ms/step
training loss : 1.1264264583587646 , training acc : 0.4287620186805725 , mcc score : 0.11026015132665634
aucroc score  : 0.5306212902069092 , f1 score     : 0.2657251

auc_roc,▁▁▁▂▄▄▆▆▇█
epoch/accuracy,▁▄▄▅▆▆█▆▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▄▃▂▂▂▂▁
epoch/val_accuracy,▁▁▂▂▅▄▇███
epoch/val_loss,█▇▆▅▄▄▃▃▂▁
fmeasure,▁▁▁▂▅▅▇█▇▇
mcc,▁▁▂▃▆▅▇███
auc_roc,0.71716
epoch/accuracy,0.48159


wandb: Agent Starting Run: 1zvrp87b with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.8
wandb: 	dropout_3: 0.6
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 256
wandb: 	fc_layer_size_2: 128
wandb: 	learning_rate: 0.0012706830419633183
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 3s 57ms/step
training loss : 0.8153708577156067 , training acc : 0.6371397972106934 , mcc score : 0.6423927545547485
aucroc score  : 0.9220414161682129 , f1 score     : 0.7386756539344788 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 58ms/step
training loss : 0.6907751560211182 , training acc : 0.7027748227119446 , mcc score : 0.6466158628463745
aucroc score  : 0.9312129616737366 , f1 score     : 0.742360532283783 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 57ms/step
training loss : 0.642311692237854 , training acc : 0.7254535555839539 , mcc score : 0.6528003811836243
aucroc score  : 0.9245562553405762 , f1 score     : 0.75791281461

auc_roc,▂▆▃█▆▆▂▁▆▆
epoch/accuracy,▁▄▆▆▇▇▇███
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▃▂▂▂▁▁▁
epoch/val_accuracy,▁▁▃▅▅▇▆▂▇█
epoch/val_loss,█▇▃▅▂▂▄▄▄▁
fmeasure,▁▂▅▄▅█▇▅▇█
mcc,▁▂▄▆▅▇▇▃▇█
auc_roc,0.93121
epoch/accuracy,0.77028


In [45]:
train_dir = "..\\enhanced\\train"
network = "densenet-121"

def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(network=network,
                              fc_layer_size_1=wandb.config.fc_layer_size_1,
                              fc_layer_size_2=wandb.config.fc_layer_size_2,
                              dropout_1=wandb.config.dropout_1,
                              dropout_2=wandb.config.dropout_2,
                              dropout_3=wandb.config.dropout_3)

        train(model=model,
              train_dir=train_dir,
              batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, 
              learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)
        
sweep_id = wandb.sweep(sweep_config, project="PCXR-DenseNet-121-Evaluation")

wandb.agent(sweep_id, function=sweep_train, count=15)

Create sweep with ID: 5k5asc7o
Sweep URL: https://wandb.ai/marcus-lim/PCXR-DenseNet-121-Evaluation/sweeps/5k5asc7o


wandb: Agent Starting Run: fmernvmk with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.8
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.6
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 512
wandb: 	fc_layer_size_2: 128
wandb: 	learning_rate: 0.0022575741070088506
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 3s 62ms/step
training loss : 0.712390124797821 , training acc : 0.6966382265090942 , mcc score : 0.3451113998889923
aucroc score  : 0.5480769276618958 , f1 score     : 0.4620908796787262 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 61ms/step
training loss : 0.6390401721000671 , training acc : 0.7291889190673828 , mcc score : 0.5753392577171326
aucroc score  : 0.807396411895752 , f1 score     : 0.703420102596283 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 61ms/step
training loss : 0.5973035097122192 , training acc : 0.7422625422477722 , mcc score : 0.5336883068084717
aucroc score  : 0.7408283948898315 , f1 score     : 0.666093587875

auc_roc,▁█▆▇▇▅▂▅▄▆
epoch/accuracy,▁▄▅▆▆▇▇█▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▂▂▂▂▁▁▁
epoch/val_accuracy,▁▆▇▇▆█▆▇▇█
epoch/val_loss,█▃▁▂▂▁▂▂▂▁
fmeasure,▁█▇▇▇▆▃▅▅▇
mcc,▁█▇▇▇▅▂▄▄▆
auc_roc,0.73772
epoch/accuracy,0.77375


wandb: Agent Starting Run: cq1fs2a3 with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.2
wandb: 	dropout_2: 0.8
wandb: 	dropout_3: 0.6
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 128
wandb: 	fc_layer_size_2: 512
wandb: 	learning_rate: 0.003350671904688304
wandb: 	optimiser: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: 6g91w3o5 with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.2
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 64
wandb: 	fc_layer_size_2: 512
wandb: 	learning_rate: 0.0024881395410352083
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 3s 60ms/step
training loss : 0.6303972005844116 , training acc : 0.7310565710067749 , mcc score : 0.444705069065094
aucroc score  : 0.7312129735946655 , f1 score     : 0.5782033801078796 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 58ms/step
training loss : 0.5394710898399353 , training acc : 0.7833511233329773 , mcc score : 0.6317824721336365
aucroc score  : 0.9039939641952515 , f1 score     : 0.7546462416648865 
Generating predictions and computing metrics for Epoch 3 
30/30 [==============================] - 2s 58ms/step
training loss : 0.5393046140670776 , training acc : 0.7814834713935852 , mcc score : 0.4339434504508972
aucroc score  : 0.6538461446762085 , f1 score     : 0.5769159793

auc_roc,▃█▁▄▂▆▃▅▄▅
epoch/accuracy,▁▆▆▆▇▇▇█▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▄▃▃▂▂▁▂▁
epoch/val_accuracy,▁▅▅▇█▇▆▇▇▇
epoch/val_loss,█▅▃▃▁▁▂▂▂▂
fmeasure,▁█▁▄▃▇▃▅▅▆
mcc,▁█▁▄▃▇▃▅▄▅
auc_roc,0.79778
epoch/accuracy,0.80496


wandb: Agent Starting Run: r57jqk9h with config:
wandb: 	batch_size: 32
wandb: 	dropout_1: 0.2
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.8
wandb: 	epochs: 10
wandb: 	fc_layer_size_1: 128
wandb: 	fc_layer_size_2: 64
wandb: 	learning_rate: 0.0033968957251172406
wandb: 	optimiser: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 4684 files belonging to 3 classes.
Using 3748 files for training.
Using 936 files for validation.
Found 1172 files belonging to 3 classes.
Generating predictions and computing metrics for Epoch 1 
30/30 [==============================] - 3s 62ms/step
training loss : 1.1219009160995483 , training acc : 0.4255602955818176 , mcc score : 0.17530114948749542
aucroc score  : 0.5600591897964478 , f1 score     : 0.3760264813899994 
Generating predictions and computing metrics for Epoch 2 
30/30 [==============================] - 2s 60ms/step
training loss : 0.9598714113235474 , training acc : 0.5474919676780701 , mcc score : 0.40259841084480286
aucroc score  : 0.7081360816955566 , f1 score     : 0.5886145234107971 


In [ ]:
train_dir = "..\\artifacts\\train"
network = "densenet-201"

def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(network=network,
                              fc_layer_size_1=wandb.config.fc_layer_size_1,
                              fc_layer_size_2=wandb.config.fc_layer_size_2,
                              dropout_1=wandb.config.dropout_1,
                              dropout_2=wandb.config.dropout_2,
                              dropout_3=wandb.config.dropout_3)

        train(model=model,
              train_dir=train_dir,
              batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, 
              learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)
        
sweep_id = wandb.sweep(sweep_config, project="PCXR-DenseNet-201-Evaluation")

wandb.agent(sweep_id, function=sweep_train, count=15)

In [ ]:
train_dir = "..\\enhanced\\train"
network = "densenet-201"

def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(network=network,
                              fc_layer_size_1=wandb.config.fc_layer_size_1,
                              fc_layer_size_2=wandb.config.fc_layer_size_2,
                              dropout_1=wandb.config.dropout_1,
                              dropout_2=wandb.config.dropout_2,
                              dropout_3=wandb.config.dropout_3)

        train(model=model,
              train_dir=train_dir,
              batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, 
              learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)
        
sweep_id = wandb.sweep(sweep_config, project="PCXR-DenseNet-201-Evaluation")

wandb.agent(sweep_id, function=sweep_train, count=15)

In [70]:
# Close WandB run:
wandb.finish()